**This Notebook has been created by Ali Hamza (9th January, 2022) to train Sentence Transformer with different Losses such as:**

> Softmax Loss

> Cusine Loss

> TripletHard Loss

> MultpleNegativesRanking Loss









In [1]:
# !pip3 install tensorflow
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('drive/sent_tans')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip3 install python-box
from box import Box

In [3]:
# !pip install sentence-transformers

In [4]:
from sentence_transformers import SentenceTransformer, SentencesDataset, losses, util
from sentence_transformers import models, losses, datasets
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import math 
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
import random
import torch

In [12]:

##### Train params  #################
def test():
    
  
  #  Run Various test suing strans_former,
  # Mostly Single sentence   ---> Classification

    cc = Box({}) 
    cc = Box({})
    cc.epoch = 3
    cc.lr = 1E-5
    cc.warmup = 100

    cc.n_sample  = 1000
    cc.batch_size=16

    cc.mode = 'cpu/gpu'
    cc.ncpu =5
    cc.ngpu= 2  
    
  ### Classifier with Cosinus Loss
    print("Classifier with Cosinus Loss ")
    sentrans_train(lossname=  "cosinus" , modelname_or_path = "/content/sample_data/results/cosinus", taskname= "classifier" , cc=cc)

  ### Classifier with Triplet Hard  Loss
    print("Classifier with Triplet Hard  Loss")
    sentrans_train( lossname = 'tripletloss', modelname_or_path = "/content/sample_data/results/triplethard", taskname = 'classifier'  , cc=cc)


   ### Classifier with Softmax Loss
    print("Classifier with Softmax Loss")
    sentrans_train( lossname = 'softmax' , modelname_or_path = "/content/sample_data/results/softmax", taskname = 'classifier'  , cc=cc)


   ### Classifier with MultpleNegativesRankingLoss Loss
    print("Classifier with Cosinus Loss")
    sentrans_train( lossname = 'MultpleNegativesRankingLoss' , modelname_or_path = "/content/sample_data/results/multpleNegativesRankingLoss",  taskname = 'classifier'  , cc=cc)
    
   ### Ranking with Cosinus Loss
   # I have written the script for evaluation in the bottom of sentrans_train() function

In [6]:
def download_dataset():
    #Check if dataset exsist. If not, download and extract  it
    nli_dataset_path = '/content/sample_data/sent_tans/AllNLI.tsv.gz'
    sts_dataset_path = '/content/sample_data/stsbenchmark.tsv.gz'

    if not os.path.exists(nli_dataset_path):
        util.http_get('https://sbert.net/datasets/AllNLI.tsv.gz', nli_dataset_path)

    if not os.path.exists(sts_dataset_path):
        util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

In [7]:
def create_train_data_for_ranking_loss(batch_size):
    
     # This function load the fake dataset if it's already existed otherwise downloads it first.
    # then Preprocess the data for MultpleNegativesRanking loss function and return it as dataloader
    
    download_dataset()
    nli_dataset_path = 'drive/sent_tans/AllNLI.tsv.gz'
    sts_dataset_path = 'drive/sent_tans/stsbenchmark.tsv.gz'
    # Read the AllNLI.tsv.gz file and create the training dataset
    logging.info("Read AllNLI train dataset")

    def add_to_samples(sent1, sent2, label):
        if sent1 not in train_data:
            train_data[sent1] = {'contradiction': set(), 'entailment': set(), 'neutral': set()}
            train_data[sent1][label].add(sent2)


    train_data = {}
    with gzip.open(nli_dataset_path, 'rt', encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            if row['split'] == 'train':
                sent1 = row['sentence1'].strip()
                sent2 = row['sentence2'].strip()

                add_to_samples(sent1, sent2, row['label'])
                add_to_samples(sent2, sent1, row['label'])  #Also add the opposite


    train_samples = []
    for sent1, others in train_data.items():
        if len(others['entailment']) > 0 and len(others['contradiction']) > 0:
            train_samples.append(InputExample(texts=[sent1, random.choice(list(others['entailment'])), random.choice(list(others['contradiction']))]))
            train_samples.append(InputExample(texts=[random.choice(list(others['entailment'])), sent1, random.choice(list(others['contradiction']))]))

    logging.info("Train samples: {}".format(len(train_samples)))
    
    # 1000 samples will be used in as train samples to keep fake dataset small
    train_dataloader = datasets.NoDuplicatesDataLoader(train_samples[:1000], batch_size=batch_size)
    return train_dataloader
    

In [8]:
def create_train_data_for_softmax_cusinus_tripletHard_loss(batch_size, lossname = 'softmax'):
    
    # This function load the fake dataset if it's already existed otherwise downloads it first.
    # then Preprocess the data for given loss function and return it as dataloader
    
    download_dataset()
    nli_dataset_path = '/content/sample_data/sent_tans/AllNLI.tsv.gz'
    sts_dataset_path = '/content/sample_data/stsbenchmark.tsv.gz'
    
    # Read the AllNLI.tsv.gz file and create the training dataset
    logging.info("Read AllNLI train dataset")

    
    label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}
    train_samples = []
    with gzip.open(nli_dataset_path, 'rt', encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            if row['split'] == 'train':
                if lossname == "softmax":
                    label_id = label2int[row['label']]
                else:
                    label_id = float(label2int[row['label']])
                    
                train_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=label_id))

    logging.info("Train samples: {}".format(len(train_samples)))
    
    # 1000 samples will be used in as train samples to keep fake dataset small
    train_dataloader = DataLoader(train_samples[:1000], shuffle=True, batch_size=batch_size)
    return train_dataloader, label2int

In [9]:
def create_train_data_for_loss(cc:dict= None , lossname = "MultpleNegativesRankingLoss"):
    
    if lossname == "MultpleNegativesRankingLoss":
        train_dataloader = create_train_data_for_ranking_loss(cc.batch_size)
        
    else:
        train_dataloader, label2int = create_train_data_for_softmax_cusinus_tripletHard_loss(cc.batch_size,lossname)
    
    return train_dataloader, label2int
        

In [11]:
def sentrans_train(modelname_or_path="",
                 taskname="classifier", 
                 lossname="cosinus",
                 train_path="train/*.csv",
                 val_path="val/*.csv",
                 metricname='cosinus',
                 dirout ="mymodel_save/",
                 cc:dict= None):
  #
  
  #  """"
  #     load a model,
  #     load a loss/task
  #     load dataset
  #     fine tuning train the model
  #     evaluate
  #     save on disk
  #     reload the model for check.
  # """

  
    cc = Box(cc)   #### can use cc.epoch   cc.lr

  # """  
  # cc = Box({})
  # cc.epoch = 3
  # cc.lr = 1E-5
  # cc.warmup = 100
  # cc.n_sample  = 1000
  # cc.batch_size=16
  # cc.mode = 'cpu/gpu'
  # cc.ncpu =5
  # cc.ngpu= 2
  # """
  

  ### load model form disk or from internet

    model = SentenceTransformer('distilbert-base-nli-mean-tokens')


 

  ###Read STSbenchmark dataset and use it as development set
    download_dataset()
    nli_dataset_path = '/content/sample_data/sent_tans/AllNLI.tsv.gz'
    sts_dataset_path = '/content/sample_data/stsbenchmark.tsv.gz'
    
    logging.info("Read STSbenchmark dev dataset")
    dev_samples = []
    with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
        
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            if row['split'] == 'dev':
                score = float(row['score']) / 5.0 #Normalize score to range 0 ... 1
                dev_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))

    dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=cc.batch_size, name='sts-dev')

    ### create  a loss
    if lossname == 'cosinus':
        train_dataloader, label2int = create_train_data_for_loss(cc , "cosinus")
        train_loss = losses.CosineSimilarityLoss(model)


    elif lossname == 'tripletloss':
        train_dataloader, label2int = create_train_data_for_loss(cc , "tripletloss")
        train_loss = losses.BatchHardTripletLoss(model=model)

    elif lossname =='softmax':
        train_dataloader, label2int = create_train_data_for_loss(cc , "softmax")
        train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=len(label2int))
  
    elif lossname == 'MultpleNegativesRankingLoss':
        train_dataloader = create_train_data_for_loss(cc , "MultpleNegativesRankingLoss")
        train_loss = losses.MultipleNegativesRankingLoss(model)

    if taskname == 'classifier':
        opt = Adam(learning_rate = cc.lr, epsilon=1e-08)
        metrics = [SparseCategoricalAccuracy("accuracy")]

        # Configure the training
        warmup_steps = math.ceil(len(train_dataloader) * cc.epoch * 0.1) #10% of train data for warm-up.  
        logging.info("Warmup-steps: {}".format(warmup_steps))
    
    
        print(len(train_dataloader))
 
        #train with multi-cpus  or multi-gpus  and save the model
        model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=cc.epoch,
          evaluation_steps= cc.n_sample,
          warmup_steps=cc.warmup,
          output_path=modelname_or_path,
          use_amp=True          #Set to True, if you want to run on GPU
          )

        print("\n******************< Training finish > ********************\n\n")

        ### show metrics

        test_samples = []
        with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
          reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
          for row in reader:
            if row['split'] == 'test':
              score = float(row['score']) / 5.0 #Normalize score to range 0 ... 1
              test_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))

        model = SentenceTransformer(modelname_or_path)
        test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=16, name='sts-test')
        test_evaluator(model, output_path=modelname_or_path)

        #### reload model
        modelname_or_path = "./results/softmax"
        model = model = SentenceTransformer(modelname_or_path)
        model.eval()
        

In [13]:
test()


Classifier with Cosinus Loss 
63


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]

Iteration:   0%|          | 0/63 [00:00<?, ?it/s]


******************< Training finish > ********************


